In [5]:
import os
import gensim
import pickle
import _pickle as cPickle

import numpy as np
import re
import string
import gensim
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [9]:


# specify the path to the folder containing the .vec files
folder_path = '/Volumes/olaaaa/Thesis_backup/bulk/tmp_downloads/word_vectors'

# create an empty dictionary to store the embeddings
embeddings_dict = {}

# loop through all the files in the folder
for file_name in os.listdir(folder_path):
    # check if the file is a .vec file
    if file_name.endswith('.vec'):
        # construct the full path to the file
        file_path = os.path.join(folder_path, file_name)
        
        # use gensim to load the embeddings from the file
        model = gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=False)
        
        # add the embeddings to the dictionary, using the file name as the key
        embeddings_dict[file_name] = model


In [15]:

# with open('embeddings_dict.pkl', 'wb') as f:
#     cPickle.dump(embeddings_dict, f, protocol=pickle.HIGHEST_PROTOCOL)


In [2]:
with open("embeddings_dict.pkl", "rb") as f:
    embeddings_dict = cPickle.load(f)

print(embeddings_dict)

{'wiki.or.vec': <gensim.models.keyedvectors.KeyedVectors object at 0x1187506a0>, 'wiki.kn.vec': <gensim.models.keyedvectors.KeyedVectors object at 0x118794280>, 'wiki.pa.vec': <gensim.models.keyedvectors.KeyedVectors object at 0x1187947c0>, 'wiki.hi.vec': <gensim.models.keyedvectors.KeyedVectors object at 0x1187948e0>, 'wiki.en.vec': <gensim.models.keyedvectors.KeyedVectors object at 0x1187942b0>, 'wiki.gu.vec': <gensim.models.keyedvectors.KeyedVectors object at 0x118794820>, 'wiki.te.vec': <gensim.models.keyedvectors.KeyedVectors object at 0x1187945b0>, 'wiki.ta.vec': <gensim.models.keyedvectors.KeyedVectors object at 0x1187943d0>, 'wiki.bn.vec': <gensim.models.keyedvectors.KeyedVectors object at 0x118794970>}


In [16]:
# create a tokenizer to split the code-mixed sentences into tokens
tokenizer = Tokenizer(filters='', lower=False)
                
def generate_embedding(sentence):
    # split the sentence into tokens
    tokens = re.findall(r"[\w']+|[^\w\s]", sentence, re.UNICODE)
    
    # create a list to store the embeddings for each token
    embeddings = []
    
    # loop through each token in the sentence
    for token in tokens:
        # check if the token is a punctuation mark
        if token in string.punctuation:
            # skip punctuation marks
            continue
        
        # check if the token is in any of the language models
        found = False
        for lang_code in embeddings_dict.keys():
            if token in embeddings_dict[lang_code]:
                # get the embedding for the token in the appropriate language model
                embedding = embeddings_dict[lang_code][token]
                
                # add the embedding to the list
                embeddings.append(embedding)
                
                # mark the token as found in a language model
                found = True
        
        # if the token was not found in any language model, skip it
        if not found:
            continue
    
    # create a 300d vector by taking the average of the embeddings
    if len(embeddings) > 0:
        embeddings = np.array(embeddings)
        vector = np.mean(embeddings, axis=0)
    else:
        vector = np.zeros(300)
    
    return vector



In [17]:
import time

# generate 1000 random code-mixed sentences
sentences = [
    'I am ગુજરાતી and தமிழ் பேசுவேன்',
    'मेरा नाम रवि है and I love ಕನ್ನಡ',
    'আমি একটি বাংলা ভাষা and నాకు తెలుగు మాట్లాడాలి',
    'My name is John and मैं हिंदी बोलता हूँ',
    'தமிழ் எனக்கு பிடித்தது and ಕನ್ನಡ ಸುಂದರ ಭಾಷೆ',
    'నేను తెలుగువాడు and আমি বাংলা বলতে পারি',
    'मी मराठी बोलतो आणि ನನಗೆ ಕನ್ನಡ ಗೊತ್ತಿದೆ',
    'আমি বাঙালি এবং నేను తెలుగు మాట్లాడుతున్నాను and தமிழ் அறிவேன்',
    'ನಾನು ಕನ್ನಡವನ್ನು ಮಾತನಾಡುತ್ತೇನೆ and मैं हिंदी बोलता हूँ'
]

# create a list to store the embeddings for each sentence
embeddings = []

# record the start time
start_time = time.time()

# loop through each sentence and generate the embedding
for sentence in sentences:
    embedding = generate_embedding(sentence)
    embeddings.append(embedding)

# record the end time
end_time = time.time()

# calculate the time taken to generate embeddings for all the sentences
time_taken = end_time - start_time

print(f'Time taken to generate embeddings for {len(sentences)} sentences: {time_taken:.2f} seconds')


Time taken to generate embeddings for 9 sentences: 0.02 seconds


(300,)

In [5]:
embedding = model['સંસ્કૃત']
print(embedding.shape)

(300,)
